In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from ast import literal_eval
import sklearn as sk
import seaborn as sb
import re, string
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

import torch
#import torchvision

In [68]:
# def preprocess_in_one(text):
#     text = text.lower()
#     text = text.strip()
#     text = re.compile('<.*?>').sub('', text) 
#     text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
#     text = re.sub('\s+', ' ', text)  
#     text = re.sub(r'\[[0-9]*\]',' ',text) 
#     text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
#     text = re.sub(r'\d',' ',text) 
#     text = re.sub(r'\s+',' ',text) 
#     return text

# def remove_stopwords(text):
#     new_text = [word for word in text.split() if word not in stopwords.words("english")]
#     return ' '.join(new_text)

# def lemmatize_text(text):
#     lemmatizer = WordNetLemmatizer()

#     # Hilfsfunktion um PosTags zu mappen
#     def get_wordnet_pos(tag):
#         if tag.startswith('J'):
#             return wordnet.ADJ
#         elif tag.startswith('V'):
#             return wordnet.VERB
#         elif tag.startswith('N'):
#             return wordnet.NOUN
#         elif tag.startswith('R'):
#             return wordnet.ADV
#         else:
#             return wordnet.NOUN
    
#     pos_tags = nltk.pos_tag(word_tokenize(text))
#     new_text = [lemmatizer.lemmatize(tag[0], get_wordnet_pos(tag[1])) for index, tag in enumerate(pos_tags)]
#     return " ".join(new_text)
    
# def combined_processing(text):
#     return lemmatize_text(remove_stopwords(preprocess_in_one(text)))

In [24]:
data = pd.read_csv("data/preprocessed_data.csv")

In [25]:
#data["preprocessed_text"] = data["text"].apply(lambda text: combined_processing(text))
data.head(20)

,Unnamed: 0,text,gender,age,topic,sign,preprocessed_text
0,0,This blog is being posted due to the fact that...,1,14,Student,Sagittarius,blog post due fact little development happen r...
1,1,So I have a big fucking interview tomorrow for...,1,15,Student,Pisces,big fucking interview tomorrow new school also...
2,2,I was reminded just now of the time Ashley and...,2,17,Student,Gemini,reminded time ashley drove kemah windows take ...
3,3,I was checking up on my cousin Dylan and Fanni...,2,23,Student,Taurus,check cousin dylan fannie wed site get married...
4,4,for the NME interview click urlLink part 1 and...,2,23,Student,Aquarius,nme interview click urllink part urllink part
5,5,Like a certain friend of mine who shant be nam...,2,14,Student,Scorpio,like certain friend mine shant name spit bfs m...
6,6,Chloe was almost lost today. SADNESS. My mom c...,2,16,Student,Libra,chloe almost lose today sadness mom come grand...
7,7,I am now officially old enough to get a permit...,1,15,Student,Capricorn,officially old enough get permit even though a...
8,8,capture the flag: sounds good me and kassovic ...,2,17,Student,Libra,capture flag sound good kassovic need back go ...
9,9,Yes School is out for summer The sun is shinin...,2,17,Student,Leo,yes school summer sun shin life great happy we...


## Gender

### Data Splitting

In [79]:
target = "gender"
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(data["preprocessed_text"], data[target], test_size=test_size,shuffle=True)

### Vectorization

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors = vectorizer.fit_transform(X_train.astype('U'))
X_test_vectors = vectorizer.transform(X_test.astype('U'))

### Machine Learning

In [82]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [83]:
lr = LogisticRegression()
nb = MultinomialNB()
rf = RandomForestClassifier(max_depth=15, n_estimators=150)
svm = SVC()

In [84]:
lr.fit(X_train_vectors, y_train)
print("LR Fitted")

nb.fit(X_train_vectors, y_train)
print("NB Fitted")

rf.fit(X_train_vectors, y_train)
print("RF Fitted")

# svm.fit(X_train_vectors, y_train)
# print("SVM Fitted")

LR Fitted
NB Fitted
RF Fitted


In [85]:
y_pred_lr = lr.predict(X_test_vectors)
y_pred_nb = nb.predict(X_test_vectors)
y_pred_rf = rf.predict(X_test_vectors)

print("----------------Logistic Regression----------------")
print(classification_report(y_test, y_pred_lr))

print("----------------Naive Bayes----------------")
print(classification_report(y_test, y_pred_nb))

print("----------------Random Forest----------------")
print(classification_report(y_test, y_pred_rf))

----------------Logistic Regression----------------
              precision    recall  f1-score   support

           1       0.70      0.80      0.75      2221
           2       0.67      0.55      0.61      1679

    accuracy                           0.69      3900
   macro avg       0.69      0.67      0.68      3900
weighted avg       0.69      0.69      0.69      3900

----------------Naive Bayes----------------
              precision    recall  f1-score   support

           1       0.66      0.90      0.77      2221
           2       0.75      0.40      0.52      1679

    accuracy                           0.68      3900
   macro avg       0.71      0.65      0.64      3900
weighted avg       0.70      0.68      0.66      3900

----------------Random Forest----------------
              precision    recall  f1-score   support

           1       0.60      0.96      0.74      2221
           2       0.73      0.14      0.24      1679

    accuracy                           0

In [86]:
confusion_matrix(y_test, y_pred_lr)

array([[1767,  454],
       [ 749,  930]], dtype=int64)

In [87]:
confusion_matrix(y_test, y_pred_nb)

array([[2004,  217],
       [1014,  665]], dtype=int64)

In [88]:
confusion_matrix(y_test, y_pred_rf)

array([[2134,   87],
       [1439,  240]], dtype=int64)

### Hyperparameter-Tuning

In [89]:
solvers = ["lbfgs", "sag", "saga", "newton-cg", "liblinear"]
penalties = ["l1", "l2", "elasticnet", "none"]
c_values = [0.001, 0.01, 0.1, 1, 10, 100]

In [90]:
c_list = []
solver_list = []
penalty_list = []
acc_list = []

for solver in tqdm(solvers):
    for penalty in penalties:
        for c in c_values:
            try:
                lr = LogisticRegression(solver=solver, penalty=penalty, C=c)
                lr.fit(X_train_vectors, y_train)
                preds = lr.predict(X_test_vectors)

                solver_list.append(solver)
                penalty_list.append(penalty)
                c_list.append(c)
                acc_list.append(accuracy_score(y_test, preds))
            except(ValueError):
                print("-----ValueError------")
            
data_log = pd.DataFrame({"Solver": solver_list, "Penalty": penalty_list, "C": c_list, "Accuracy": acc_list})

  0%|          | 0/5 [00:00<?, ?it/s]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


 20%|██        | 1/5 [00:06<00:24,  6.08s/it]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


 40%|████      | 2/5 [00:13<00:20,  6.86s/it]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


 60%|██████    | 3/5 [02:39<02:20, 70.38s/it]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


100%|██████████| 5/5 [02:55<00:00, 35.06s/it]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


In [91]:
data_log.sort_values(by="Accuracy", ascending=False)

,Solver,Penalty,C,Accuracy
33,saga,l2,1.000,0.691538
15,sag,l2,1.000,0.691538
3,lbfgs,l2,1.000,0.691538
63,liblinear,l2,1.000,0.691538
45,newton-cg,l2,1.000,0.691538
...,...,...,...,...
12,sag,l2,0.001,0.569487
30,saga,l2,0.001,0.569487
25,saga,l1,0.010,0.569487
24,saga,l1,0.001,0.569487


In [92]:
data.to_csv("results/results_gender.csv")

## Sternzeichen

In [69]:
target = "sign"
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(data["preprocessed_text"], data[target], test_size=test_size,shuffle=True)

In [70]:
vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors = vectorizer.fit_transform(X_train.astype('U'))
X_test_vectors = vectorizer.transform(X_test.astype('U'))

In [71]:
lr = LogisticRegression()
nb = MultinomialNB()
rf = RandomForestClassifier(max_depth=15, n_estimators=150)
svm = SVC()

In [72]:
lr.fit(X_train_vectors, y_train)
print("LR Fitted")

nb.fit(X_train_vectors, y_train)
print("NB Fitted")

rf.fit(X_train_vectors, y_train)
print("RF Fitted")

LR Fitted
NB Fitted
RF Fitted


In [73]:
y_pred_lr = lr.predict(X_test_vectors)
y_pred_nb = nb.predict(X_test_vectors)
y_pred_rf = rf.predict(X_test_vectors)

print("----------------Logistic Regression----------------")
print(classification_report(y_test, y_pred_lr))

print("----------------Naive Bayes----------------")
print(classification_report(y_test, y_pred_nb))

print("----------------Random Forest----------------")
print(classification_report(y_test, y_pred_rf))

----------------Logistic Regression----------------
              precision    recall  f1-score   support

    Aquarius       0.15      0.10      0.12       277
       Aries       0.21      0.24      0.23       369
      Cancer       0.16      0.26      0.20       362
   Capricorn       0.24      0.11      0.15       257
      Gemini       0.26      0.17      0.21       315
         Leo       0.20      0.15      0.17       314
       Libra       0.23      0.22      0.23       325
      Pisces       0.22      0.24      0.23       365
 Sagittarius       0.35      0.20      0.25       310
     Scorpio       0.19      0.23      0.21       367
      Taurus       0.16      0.18      0.17       315
       Virgo       0.17      0.26      0.21       324

    accuracy                           0.20      3900
   macro avg       0.21      0.20      0.20      3900
weighted avg       0.21      0.20      0.20      3900

----------------Naive Bayes----------------
              precision    recall  f1

### Hyperparameter Tuning

In [74]:
solvers = ["lbfgs", "sag", "saga", "newton-cg", "liblinear"]
penalties = ["l1", "l2", "elasticnet", "none"]
c_values = [0.001, 0.01, 0.1, 1, 10, 100]

In [75]:
c_list = []
solver_list = []
penalty_list = []
acc_list = []

for solver in tqdm(solvers):
    for penalty in penalties:
        for c in c_values:
            try:
                lr = LogisticRegression(solver=solver, penalty=penalty, C=c)
                lr.fit(X_train_vectors, y_train)
                preds = lr.predict(X_test_vectors)

                solver_list.append(solver)
                penalty_list.append(penalty)
                c_list.append(c)
                acc_list.append(accuracy_score(y_test, preds))
            except(ValueError):
                print("-----ValueError------")
            
data_log = pd.DataFrame({"Solver": solver_list, "Penalty": penalty_list, "C": c_list, "Accuracy": acc_list})

  0%|          | 0/5 [00:00<?, ?it/s]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


 20%|██        | 1/5 [01:13<04:53, 73.40s/it]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


 40%|████      | 2/5 [01:47<02:31, 50.40s/it]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


 60%|██████    | 3/5 [12:54<11:03, 331.95s/it]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


100%|██████████| 5/5 [25:23<00:00, 304.79s/it]

-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------
-----ValueError------


In [76]:
data_log.sort_values(by="Accuracy", ascending=False)

,Solver,Penalty,C,Accuracy
28,saga,l1,10.000,0.209487
64,liblinear,l2,10.000,0.209231
65,liblinear,l2,100.000,0.207692
4,lbfgs,l2,10.000,0.207436
58,liblinear,l1,10.000,0.206154
...,...,...,...,...
42,newton-cg,l2,0.001,0.092821
24,saga,l1,0.001,0.092821
25,saga,l1,0.010,0.092821
30,saga,l2,0.001,0.092821


In [77]:
data.to_csv("results/results_sternzeichen.csv")